In [2]:
# Imports
!pip install -q datasets emoji
from datasets import load_dataset
import re
import emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 7.8 MB/s eta 0:00:00


In [3]:
# Cargar el dataset
database = "amaiaruvi/news_racist_comments_spanish"
dataset = load_dataset(database)
dataset

Generating train split:   0%|          | 0/3005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/438 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/851 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 3005
    })
    validation: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 438
    })
    test: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 851
    })
})

In [4]:
# Eliminar saltos de línea y espacios repetidos
def delete_spaces(comment):
  spaces_pattern = r'[\n\r]+|\s+'
  return re.sub(spaces_pattern, ' ', comment)

In [5]:
# Convertir todo a minúsculas
def lower_text(comment):
  return comment.lower()

In [6]:
# Eliminar URL
def delete_urls(comment):
  url_pattern = r'http[s]?://\S+'
  return re.sub(url_pattern, '', comment)

In [7]:
# Eliminar consonantes repetidas y puntos suspensivos
def delete_repeated_consonants(comment):
  repeated_consonant_pattern = r'([^aeiou\s\r\n0-9])\1{1,}'
  def replace(match):
      char = match.group(1)
      if char in 'rcnl':
          return char * 2
      else:
          return char

  return re.sub(repeated_consonant_pattern, replace, comment, flags=re.IGNORECASE)

In [8]:
# Dejar como máximo 2 vocales iguales contiguas
def delete_repeated_vowels(comment):
    repeated_vowels_pattern = r'([aeiouAEIOU])\1{2,}'
    return re.sub(repeated_vowels_pattern, r'\1\1', comment, flags=re.IGNORECASE)

In [9]:
# Eliminar acentos no empleados en Español
def delete_accents (comment):
  comment = re.sub(r"[àâãäå]", "a", comment)
  comment = re.sub(r"ç", "c", comment)
  comment = re.sub(r"[èêë]", "e", comment)
  comment = re.sub(r"[ìîï]", "i", comment)
  comment = re.sub(r"[òôõö]", "o", comment)
  comment = re.sub(r"[ùû]", "u", comment)
  comment = re.sub(r"[ýÿ]", "y", comment)
  return comment

In [10]:
# Eliminar caracteres inusuales
def delete_characters(comment):
  special_characters = r'[ºª|·~¬\^`[\]¨´#\\\'\(\)*\<>_]'
  return re.sub(special_characters, '', comment)

In [11]:
# Eliminar otros caracteres inusuales
def delete_specific_characters(comment):
    specific_chars_pattern = r'[«»✈ºø♀♂€🇮±°ª‘’“”🇱]'
    return re.sub(specific_chars_pattern, '', comment)

In [12]:
# Eliminar emoticonos
def delete_emoticons(comment):
    return emoji.replace_emoji(comment, replace='')

In [13]:
# Eliminar caracteres árabes
def delete_arabic_letters(text):
    arabic_pattern = r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]'
    return re.sub(arabic_pattern, '', text)

In [14]:
# Escribir todas las risas de la misma manera
def unify_laughs (comment):
  laugh_pattern = r"\b(a*ha+h[ha]*|o?l+o+l+[ol]*|x+d+[x*d*]*|a*ja+[j+a+]+|j+e+j+[ej]*|j+s+j+[sj]*|j+i+j+[ij]*)\b"
  return re.sub(laugh_pattern, 'jaja', comment, flags=re.IGNORECASE)

In [15]:
def delete_extra (comment):
  return re.sub(r'\bmikeldi\b', '', comment, flags=re.IGNORECASE)

In [16]:
def preprocess_comment(comment):
  comment = delete_spaces(comment)
  comment = lower_text(comment)
  comment = delete_urls(comment)
  comment = delete_repeated_consonants(comment)
  comment = delete_repeated_vowels(comment)
  comment = delete_accents(comment)
  comment = delete_characters(comment)
  comment = delete_specific_characters(comment)
  comment = delete_emoticons(comment)
  comment = delete_arabic_letters(comment)
  comment = unify_laughs(comment)
  comment = delete_extra(comment)
  return comment

In [17]:
preprocessed_data = dataset.map(lambda ex: {
    "comment": preprocess_comment(ex["comment"])
})

Map:   0%|          | 0/3005 [00:00<?, ? examples/s]

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

Map:   0%|          | 0/851 [00:00<?, ? examples/s]

In [18]:
unify_laughs("ajaj ajajajaj jejeje jsjsjsjs hahaha lollol jijijij ajaj ajajajajaja")

'jaja jaja jaja jaja jaja jaja jaja jaja jaja'

In [39]:
dataset['train']['comment'][1092:1097]

['No sabes por dónde te pega el aire no dices más que sandeces',
 '🤣🤣 Acomplejados y patéticos. si no está para jugar que se quede en su casa.',
 'JAJAJAJAJAJAA\nPerdonad, pero no he podido evitarlo\nGrande Europa.',
 'A los palestinos no los quieren ni sus propios colegas etnicos, religiosos y demas. Porque no les ayudan los paises riquisismos del golfo ?? porque les tiene que ayudar Europa ?? A pensar.',
 'A algún país del primer mundo (o corporación internacional) no le venía bien y se lo han quitado de en medio. Ahora pondrán una marioneta que les favorezca, y el mundo sigue girando...']

In [40]:
preprocessed_data = dataset.map(lambda ex: {
    "comment": preprocess_comment(ex["comment"])
})
preprocessed_data['train']['comment'][1092:1097]

['no sabes por dónde te pega el aire no dices más que sandeces',
 ' acomplejados y patéticos. si no está para jugar que se quede en su casa.',
 'jaja perdonad, pero no he podido evitarlo grande europa.',
 'a los palestinos no los quieren ni sus propios colegas etnicos, religiosos y demas. porque no les ayudan los paises riquisismos del golfo ? porque les tiene que ayudar europa ? a pensar.',
 'a algún país del primer mundo o corporación internacional no le venía bien y se lo han quitado de en medio. ahora pondrán una marioneta que les favorezca, y el mundo sigue girando.']